In [82]:
import cv2
import time
import numpy as np
import math
import HandTrackingModule as htm
import os

In [51]:
wCam, hCam = 640, 480

In [64]:
detector = htm.handDetector(detectionCon=0.7)


In [71]:
cap = cv2.VideoCapture(0)
cap.set(5, wCam)
cap.set(5, hCam)
pTime = 0

volRange = [0, 100]
minVol = volRange[0]
maxVol = volRange[1]

while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList, bbox = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        #print(lmList)
        #print(lmList[4], lmList[8])

        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)

        length = math.hypot(x2 - x1, y2 - y1)
        print(length)

        def set_volume(volume):
            os.system(f"osascript -e 'set volume output volume {volume}'")
        set_volume(100)

        vol = np.interp(length, [50, 300], [minVol, maxVol])
        volBar = np.interp(length, [50, 300], [400, 150])
        volPer = np.interp(length, [50, 300], [0, 100])
        print(int(length), vol)

        if length < 50:
            cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_COMPLEX, 
                1, (255, 0, 0), 3)
    cv2.imshow("Img", img)
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

208.73428084528905
230.76611536358627
226.10616975217638
215.7127719909046
234.25200105868893
228.97379762758882
230.7639486574972
234.7977853387889
231.2444593930847
235.25518060183074
238.83885781003056
244.18026128252055
246.4163143949686
240.160363090998
238.88281646028875
239.1359446005556
236.22235287965447
235.27430798963155
230.8462691922917
230.8462691922917
229.2705825002414
231.79516819813134
232.74449510138794
231.82105167564055
230.9328906847182
231.22499864850255
226.15260334561705
224.54398232862977
227.40492518852795
226.10838109190027
225.78751072634643
221.79495034828903
224.01785643113362
237.36048533822978
249.94599416673995
234.68276459936294
150.65855435387664
44.14748010928823
26.570660511172846
28.160255680657446
22.825424421026653
18.788294228055936
24.08318915758459
44.181444068749045
98.48857801796105
190.9607289470796
231.07790893982056
240.63457773146402
238.27085428142487
241.1389640850271
241.1389640850271
243.0164603478538
241.6464359348178
241.646435934

In [ ]:
def set_volume(volume):
    os.system(f"osascript -e 'set volume output volume {volume}'")
set_volume(100)